In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.agents import initialize_agent, AgentType, Tool
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
from utils import load_json
import os
from LegalPrincipleExtractorTool import LegalPrincipleExtractor
from CitationExtractorTool import CitationExtractor
import dotenv
import os
import time
from tqdm import tqdm
from utils import load_json, save_json
dotenv.load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [21]:
from shared_memory import SharedMemory


data_path = '/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/'
df1_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_1.csv'
df4_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_4.csv'
df1 = pd.read_csv(df1_path)['file_path'].to_list()
df4 = pd.read_csv(df4_path)['file_path'].to_list()

case_path = data_path + df4[0]
case = load_json(case_path)

shared_memory = SharedMemory(case_path)

In [12]:
from LegalPrincipleExtractorTool import LegalPrincipleExtractor
from CitationExtractorTool import CitationExtractor
from legalHelperTool import LegalHelper
from FactsReaderTool import CaseFactsReader
from LawReaderTool import CaseLawReader


llm = ChatOpenAI(
        temperature=0,
        model_name="gpt-4o-mini",
        openai_api_key=openai_api_key
    )
    

tools = [
        Tool(
            name='facts_reader',
            func=CaseFactsReader()._run,
            description="Load and return the case facts."
        ),
        Tool(
            name='law_reader',
            func=CaseLawReader()._run,
            description="Load and return the case law."
        ),
        Tool(
            name='legal_principle_extractor',
            func=LegalPrincipleExtractor()._run,
            description="Extract key legal principles from the case file."
        ),
        Tool(
            name='citation_extractor',
            func=CitationExtractor()._run,
            description="Extract citations from the case file."
        ),
        Tool(
            name='legal_helper',
            func=LegalHelper()._run,
            description="""A tool to answer specific legal questions about ECHR jurisprudence and legal principles. 
    Use this tool to ask focused questions about legal concepts, doctrines, or interpretations that will help  analyze the case. 
    The tool do not have access to the case file.
    Do NOT ask this tool to determine if a case is key - instead ask specific questions  about legal elements you identify from the principles and citations.
    To use this tool you must provide [file_path, question] as input.
    """
        )
    ]


    # Simplified prompt - no need to handle text passing
    # Construct the ReAct agent
    # Get the prompt to use - you can modify this!
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools,prompt)


/Users/ahmed/hackatum24/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [23]:
def get_prompt(case_path):
    prompt = f"""
You are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your role is to conduct an exhaustive legal analysis, akin to writing a detailed case commentary.
Your task is to classify the case at {case_path} into one of the following categories:

KEY CASE:
- Makes a significant contribution to the development, clarification, or modification of case law.
- Establishes new legal principles or substantially modifies existing ones.
- Has broad implications beyond the immediate case.

NOT KEY CASE:
- Applies existing case law without significant contributions to legal development.
- Demonstrates limited implications beyond the immediate dispute.


ANALYSIS WORKFLOW:

-Extract Case Facts and Law
    - Load the case facts and law from the file to understand the context and background.

- Identify Core Legal Issues
    - Determine the key legal questions and associated ECHR articles raised in the case.

- Extract and Analyze Legal Principles
    - Use the legal_principle_extractor tool to extract key principles.
    - Evaluate whether these principles:
        - Develop or clarify existing legal doctrines.
        - Balance competing rights or obligations in novel ways.

- Analyze Cited Precedents
    - Use the citation_extractor tool to extract citations.
    - Investigate:
        - How the case builds upon, clarifies, or diverges from cited precedents.
        - Whether it resolves ambiguities or tensions in prior case law.

- Investigate Novelty
    - Use the extracted principles and citations to determine:
        - Whether the case introduces new legal principles or interpretations.
        - Whether it refines or extends existing doctrines in meaningful ways.

- Assess Precedential Impact
    - Evaluate the case’s potential influence on future jurisprudence:
        -  Does it provide new guidance for interpreting ECHR articles?
        - How frequently is it cited (or could be cited) in subsequent cases?

- Compare with Similar Cases
    - Compare the case to other landmark judgments in the same area of law to assess its distinct contribution (if any).

- Evaluate Broader Implications
    - Consider the case's practical impact:
        - Does it affect state obligations, human rights practices, or jurisprudence in a meaningful way?
        - Does it resolve issues prominent at the time of its ruling?

- Iterative Questioning
    - After each step, ask follow-up questions to deepen your analysis. Examples include:
        - "What ambiguities in cited cases does this case address?"
        - "Does this case set new thresholds for interpreting ECHR rights?"
        - "What practical implications does this case have for future disputes?"

Decision Support Framework
- Summarize findings using a justification grid:
    - Novelty: Identify specific legal developments or clarifications.
    - Practical Impact: Evaluate its influence on future cases or legal practice.
    - Precedential Role: Assess how the case relates to and builds upon prior jurisprudence.

    
Decision and Explanation
- Once your analysis is complete, classify the case as KEY CASE or NOT KEY CASE based on the following guidelines:
- If it IS a Key Case:
    - Identify the specific, novel legal development introduced.
    - Explain why existing jurisprudence was insufficient to address the legal issues.
    - Describe precisely how this case advances ECHR law.

- If it is NOT a Key Case:
    - Explain why the case falls within existing jurisprudence.
    - Highlight its lack of novel contributions, limited implications, or fact-specific nature.   

Important Notes
- Be Rigorous: Each step should inform deeper follow-up questions. Avoid surface-level analysis.
- Focus on Legal Development: Simply applying established principles to new facts does NOT make a case key.
- Avoid Overgeneralization: A case involving multiple articles or issues is NOT automatically key; assess its broader impact and novelty.

Your goal is to produce a comprehensive, systematic, and well-justified analysis of the case.
    """
    return prompt


#/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-220960.json

prompt = get_prompt(case_path)
agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
I need to begin by extracting the case facts and law from the provided file to understand the context and background of the case. This will help me identify the core legal issues and the relevant ECHR articles involved.

Action: facts_reader  
Action Input: /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-229412.json  3. The list of applicants and the relevant details of the applications are set out in the appended table.
4. The applicants complained of the unlawful detention (deprivation of liberty). Some applicants also raised other complaints under the provisions of the Convention or Protocols to it.I have extracted the case facts, which indicate that the applicants are complaining about unlawful detention and potentially other complaints under the ECHR provisions. This gives me a foundational understanding of the case context. Next, I need to extract the relevant law to see how the legal framework applies to these facts.

Action: law_reade

{'input': '\nYou are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your role is to conduct an exhaustive legal analysis, akin to writing a detailed case commentary.\nYour task is to classify the case at /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-229412.json into one of the following categories:\n\nKEY CASE:\n- Makes a significant contribution to the development, clarification, or modification of case law.\n- Establishes new legal principles or substantially modifies existing ones.\n- Has broad implications beyond the immediate case.\n\nNOT KEY CASE:\n- Applies existing case law without significant contributions to legal development.\n- Demonstrates limited implications beyond the immediate dispute.\n\n\nANALYSIS WORKFLOW:\n\n-Extract Case Facts and Law\n    - Load the case facts and law from the file to understand the context and background.\n\n- Identify Core Legal Issues\n    - Determine the key legal questions and ass

In [24]:
data_path = '/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/'
df1_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_1.csv'
df4_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_4.csv'
df1 = pd.read_csv(df1_path)['file_path'].to_list()
df4 = pd.read_csv(df4_path)['file_path'].to_list()

case_path = data_path + df1[45]

#/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-220960.json

prompt = get_prompt(case_path)
agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
I need to begin by extracting the case facts and law from the provided file to understand the context and background of the case. This will help me identify the core legal issues and the relevant ECHR articles involved.

Action: facts_reader  
Action Input: /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-202621.json  1. The applicants were born in 1981 and 1978 respectively and live as a couple in R. They were represented by Ms Diana-Elena Dragomir, a lawyer practising in Bucharest.
2. The Government were represented by their Agent, most recently Ms O.F. Ezer, of the Ministry of Foreign Affairs.
3. The facts of the case, as submitted by the parties, may be summarised as follows.
BACKGROUND OF THE CASE
BACKGROUND OF THE CASE
4. In 2012 the anti-corruption prosecutor s office attached to the High Court of Cassation and Justice ( the prosecutor s office and the HCCJ ) started a criminal investigation into alleged acts of corruption committed by 

{'input': '\nYou are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your role is to conduct an exhaustive legal analysis, akin to writing a detailed case commentary.\nYour task is to classify the case at /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-202621.json into one of the following categories:\n\nKEY CASE:\n- Makes a significant contribution to the development, clarification, or modification of case law.\n- Establishes new legal principles or substantially modifies existing ones.\n- Has broad implications beyond the immediate case.\n\nNOT KEY CASE:\n- Applies existing case law without significant contributions to legal development.\n- Demonstrates limited implications beyond the immediate dispute.\n\n\nANALYSIS WORKFLOW:\n\n-Extract Case Facts and Law\n    - Load the case facts and law from the file to understand the context and background.\n\n- Identify Core Legal Issues\n    - Determine the key legal questions and ass

In [9]:
case_path = data_path + df1[45]
case_path

'/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-202621.json'

In [3]:
def get_prompt(case_file_path):

    prompt = f"""
    You are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your analysis must be precise and methodical.

    IMPORTANT DISTINCTION ABOUT KEY CASES:
    A key case is one that develops ECHR jurisprudence in a meaningful way. This happens when a case either:
    - Creates new legal principles
    - Significantly modifies existing principles
    - Extends principles to novel situations not previously considered
    - Synthesizes existing principles in an innovative way

    Simply applying established principles correctly, even in serious human rights cases, does NOT make it a key case.

    case_file_path: {case_file_path}

    To make this determination, follow these analytical steps:

    1. First, understand the legal principles involved:
    Thought: I need to identify the key legal principles and ECHR articles at play.
    Action: legal_principle_extractor
    Action Input: {case_file_path}

    2. Then, analyze how precedents are used:
    Thought: I need to examine how this case uses previous ECHR jurisprudence.
    Action: citation_analysis
    Action Input: {case_file_path}

    3. After getting both analyses, evaluate:
    - How does this case use the cited precedents?
    - Is it merely following them?
    - Is it creating something new?
    - Is it extending principles to new situations?

    Remember: The significance of human rights issues or the importance of the subject matter does NOT determine key case status. Focus solely on how the case develops ECHR jurisprudence.

    Format your analysis as:
    Thought: [Your specific reasoning at each step]
    Action: [Tool to use]
    Action Input: [File path]
    Observation: [Tool output]
    Thought: [Analysis of the observation]
    ... [Continue until conclusion reached]
    Final Answer: [Clear yes/no on key case status with specific explanation of how it does/doesn't develop jurisprudence]

    You don't need to provide the case text - the tools will access it automatically.
    """
    return prompt


In [6]:
kc_input_dir = '../../kc_classification_data/125_kc_2018_2024/'
not_kc_input_dir = '../../kc_classification_data/125_notkc_2018_2024/'

kc_input_cases = os.listdir(kc_input_dir)
not_kc_input_cases = os.listdir(not_kc_input_dir)

output_dir = './results/'

len(kc_input_cases), len(not_kc_input_cases)

(125, 125)

In [7]:
import os
case_path = os.path.join(kc_input_dir, kc_input_cases[0])
case_data = load_json(case_path)
case_facts = case_data['case_text']['THE FACTS']


In [6]:
prompt = get_prompt(case_path)
agent = create_legal_case_agent(openai_api_key, case_path)
response = agent.run(prompt)


/var/folders/dl/d24xg2c11gx7r5rs5b6kd_780000gn/T/ipykernel_28997/3354856887.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/dl/d24xg2c11gx7r5rs5b6kd_780000gn/T/ipykernel_28997/3354856887.py:42: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  return initialize_agent(
/var/folders/dl/d24xg2c11gx7r5rs5b6kd_780000gn/T/ipykernel_28997/2026218400.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be re



> Entering new AgentExecutor chain...
Thought: I need to identify the key legal principles and ECHR articles at play in the case to understand its potential impact on ECHR jurisprudence.

Action: legal_principle_extractor
Action Input: ../../kc_classification_data/125_kc_2018_2024/001-187188.json

/Users/ahmed/Desktop/msc-24/TND/key_case_binary_classification/react_1/LegalPrincipleExtractorTool.py:43: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm.predict(prompt)



Observation: In analyzing the case facts provided, several key legal principles emerge, particularly concerning the balance between freedom of expression and the protection of religious beliefs. These principles are primarily related to Articles 9 and 10 of the European Convention on Human Rights (ECHR). Below is a structured analysis mapping these principles to the relevant ECHR Articles:

### Key Legal Principles

1. **Freedom of Expression (Article 10 ECHR)**
   - **Principle**: Article 10 protects the right to freedom of expression, including the freedom to hold opinions and to receive and impart information and ideas without interference by public authority.
   - **Application**: The applicant argued that her statements were protected under Article 10, as they were expressions of opinion and criticism of religious practices, which are permissible even if they offend, shock, or disturb.
   - **Balancing**: The courts had to balance this right against the need to protect the rights

In [13]:
import os
case_path = os.path.join(not_kc_input_dir, not_kc_input_cases[0])
case_data = load_json(case_path)
case_facts = case_data['facts']


prompt = get_prompt(case_path)
agent = create_legal_case_agent(openai_api_key, case_path)
response = agent.run(prompt)




> Entering new AgentExecutor chain...
I need to identify the key legal principles and ECHR articles at play in the case.  
Action: legal_principle_extractor  
Action Input: ../../kc_classification_data/125_notkc_2018_2024/001-186776.json  
Observation: **KEY LEGAL PRINCIPLES:**

1. **Right to Life (Article 2 of the ECHR):**
   - **Principle:** Article 2 of the ECHR protects the right to life and imposes a duty on the state to conduct an effective investigation into deaths resulting from the use of force.
   - **Application:** The applicant's son was killed during the events leading to the fall of the communist regime in Romania. The state's obligation under Article 2 is engaged, requiring an effective investigation into the circumstances of the death. The ongoing investigations and the reopening of the case for crimes against humanity reflect the state's continuing duty to fulfill this obligation.
   - **Balancing/Conflict:** The principle of effective investigation under Article 2 m

In [14]:
response

'No, this case is not a key case in ECHR jurisprudence. It applies established principles regarding the need for effective investigations into state violence and does not create, significantly modify, extend, or synthesize legal principles in a novel way.'